In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import pylab
import scipy.stats as ss
import csv
import numpy as np
import sklearn
import glob
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

### CONVERTIR PACIENTES (los 1794 csv de la carpeta pacientes) A UNA LISTA Y LA LISTA A CSV

In [2]:
import os
contenido = os.listdir('C:/Users/ywen2/Desktop/URJC/TFG/TFG_Progresion_Diabetes/GitHub/TFG/ddbb/pacientes')

In [3]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
# Se que es redundante crear dos listas, pero es la única forma de trabajar con PANDAS, debo revisarlo!!!!!!!!!!!
lista_id_indep =  [item.replace(".csv", "") for item in contenido]
# Mostrar el archivo csv_files, el cual es una lista de nombres
#print(lista_id_indep)
pacientes_independientes = pd.DataFrame(lista_id_indep, columns=["Pacientes"])
pacientes_independientes.to_csv('pacientes_indep.csv', index=False)
df_independientes = pd.read_csv('pacientes_indep.csv').fillna(0)
pacientes_independientes = df_independientes.iloc[:,0]
pacientes_independientes = list(pacientes_independientes)

### CREO UNA LISTA A PARTIR DEL CSV PACIENTES PROGRESORES

In [4]:
with open('pacientes_progresores.csv', 'r') as archivo:
    leer = csv.reader(archivo)
    lista = list(leer)
    lista.pop(0)
    #print(lista)

In [5]:
df = pd.DataFrame (lista,columns=['Historia','Progresion'])
print (df)

     Historia Progresion
0      100220          1
1      100257          0
2       10045          0
3      100505          0
4      100598          0
...       ...        ...
1642    99582          0
1643     9970          0
1644    99828          1
1645    99835          0
1646     9985          0

[1647 rows x 2 columns]


In [6]:
df_progresores = pd.read_csv('pacientes_progresores.csv').fillna(0)
pacientes_progresores = df_progresores.iloc[:,0]
#print(pacientes_progresores)
pacientes_progresores = list(pacientes_progresores)
#print(pacientes_progresores)

### COMPARO AMBAS LISTAS Y ELIMINO PACIENTES NO COINCIDENTES Y CREO EL CSV CON AQUELLOS PACIENTES QUE COINCIDEN EN AMBAS LISTAS

In [7]:
comparacion = []
 
for item in pacientes_independientes:
    if item in pacientes_progresores:
        comparacion.append(item)

In [8]:
df_procesados = pd.DataFrame(comparacion, columns=["Historia"])
print(df_procesados)
df_procesados.to_csv('pacientes_procesados.csv', index=False)

      Historia
0       100220
1       100257
2        10045
3       100505
4       100598
...        ...
1642     99582
1643      9970
1644     99828
1645     99835
1646      9985

[1647 rows x 1 columns]


## AHORA QUIERO COMPARAR EL CSV PACIENTES PROGRESORES CON PACIENTES PROCESADOS Y EN AQUELLOS PACIENTES QUE COINCIDAN AÑADIR AL CSV  PACIENTES PROCESADOS LA ETIQUETA 1 o 0 QUE ACOMPAÑA A CADA NUMERO DE HISTORIA EN EL CSV PACIENTES PROGRESORES . NO SE HACERLO :(

In [9]:
file1 = df['Historia'].astype(int)

In [10]:
file2 = df_procesados['Historia'].astype(int)

In [11]:
mergedStuff = pd.merge(file1, file2, how='inner', indicator = 'union')
print(mergedStuff)

contador = 0
for i in mergedStuff['union']:
    if i != 'both':
        contador += 1
        
print(contador)

      Historia union
0       100220  both
1       100257  both
2        10045  both
3       100505  both
4       100598  both
...        ...   ...
1642     99582  both
1643      9970  both
1644     99828  both
1645     99835  both
1646      9985  both

[1647 rows x 2 columns]
0


#### EN EL FICHERO PACIENTES_PROGRESORES TENEMOS TODOS LOS PACIENTES A ESTUDIAR

## ESCRITO POR OSCAR

1. Lista de csv de los nombres de ficheros HECHO
1. Ver coincidencias de lista con progresión y lista de ficheros HECHO
1. Generar un dataframe con n_historia y con variable targer (progresion) NOS SIRVE PACIENTES_PROCESADOS

df['id'] #columna de identificación
df['progresion_diab'] = #target

df lista de pacientes que tenemos la variable y y tenemos datos.

1. Si estamos con la solución ajustar regresión polinómica

 1.pinta 5 pacientes progresores y cinco no progresores

pat_id = 5050

Plots de variables vs revisiones


X_prog = list de matrices(mxn) filas= revisiones, columnas= variables, 



X_no_prog


In [ ]:
def get_pat(idd,path='C:/Users/ywen2/Desktop/URJC/TFG/TFG_Progresion_Diabetes/GitHub/TFG/ddbb/pacientes/'):
    
    file = df.values.tolist()
    paciente = pd.read_csv(path + idd+'.csv')
    for i in file:
        if i[0] == idd:
            print('El paciente presenta la siguiente etiqueta:', i[1])
    return(paciente)
    
    #lee fichero de ese id
    

In [ ]:
'''def get_pat(idd,path='C:/Users/ywen2/Desktop/URJC/TFG/TFG_Progresion_Diabetes/GitHub/TFG/ddbb/pacientes/'):
    
    file = df.values.tolist()
    paciente = pd.read_csv(path + idd+'.csv')
    for i in df.index: 
        if df['Progresion'][i] == '1':
            print('El paciente:', df['Historia'][i], 'presenta la siguiente etiqueta:', df['Progresion'][i])
    return(paciente)'''

In [ ]:
for i in df.index:
    if df['Progresion'][i] == '1':
        print (df['Historia'][i])

In [ ]:
paciente = get_pat('100220')
paciente1 = get_pat('15015')
paciente2 = get_pat('13563')
paciente3 = get_pat('153504')
paciente4 = get_pat('224674')
paciente5 = get_pat('9985')
paciente6 = get_pat('5526')
paciente7 = get_pat('7658')
paciente8 = get_pat('12170')
paciente9 = get_pat('390511')

In [ ]:
X = np.matrix(paciente)
X1 = np.matrix(paciente1)
X2 = np.matrix(paciente2)
X3 = np.matrix(paciente3)
X4 = np.matrix(paciente4)
X5 = np.matrix(paciente5)
X6 = np.matrix(paciente6)
X7 = np.matrix(paciente7)
X8 = np.matrix(paciente8)
X9 = np.matrix(paciente9)

#X = np.transpose(X)
#print(X)

In [ ]:

print(X5.shape)
print(X6.shape)
print(X7.shape)
print(X8.shape)
print(X9.shape)

In [ ]:
def get_matrix():
    progresores = []
    no_progresores = []
    for i in df.index:
        if df['Progresion'][i] == '1':
             progresores.append((df['Historia'][i]))
        else:
            no_progresores.append((df['Historia'][i]))
   


In [12]:
progresores = []
no_progresores = []
fila = []
for i in df.index:
    if df['Progresion'][i] == '1':
         progresores.append((df['Historia'][i]))
    else:
        no_progresores.append((df['Historia'][i]))
        
for paciente in progresores:
        pacientes = pd.read_csv('C:/Users/ywen2/Desktop/URJC/TFG/TFG_Progresion_Diabetes/GitHub/TFG/ddbb/pacientes/' + paciente+'.csv')
        matrices = np.matrix(pacientes)
        print(matrices)
        #print(len(matrices))
        filas = matrices.shape[0]
        fila.append(filas)
        fila_max = np.max(fila)
        #matriz_array = np.array(matrices)
        #print(matriz_array)


        X_features = np.zeros((len(progresores) ,col_max))
        print(X_features.shape)
        X_features[:,:] = np.nan
        for i in columna:
            for j in range(len(progresores)):
                X_features[j,:i] = matrices[i][:,4].flatten()'''

[[50 78 152 33.8 0.6 nan 61.0 126.0 67 53.0 61 41.0 9.46 102.0 nan nan 95
  nan nan nan 129 92 '2005-11-07']
 [50 73 152 31.6 0.8 1.0 55.0 141.0 92 nan 56 141.0 13.0 83.0 nan nan 88
  nan 3.1 nan 107 70 '2006-05-30']
 [51 71 152 30.7 0.9 nan 63.0 129.0 96 30.0 37 22.0 nan 85.0 3.1 12.7 98
  nan 3.1 nan 127 88 '2006-11-22']
 [51 73 152 31.6 0.6 0.84 64.0 130.0 77 30.0 37 23.0 4.9 55.0 nan nan 91
  nan 3.0 nan 140 88 '2007-05-24']
 [52 78 152 33.8 0.7 0.86 74.0 94.0 56 36.0 48 59.0 21.81 96.0 nan nan 83
  nan 3.0 nan 136 86 '2007-11-20']
 [52 78 152 33.8 0.7 0.92 68.0 91.0 74 51.0 76 58.0 12.63 80.0 nan nan 88
  nan 3.36 nan 157 95 '2008-05-29']
 [53 81 152 35.1 0.7 nan 66.0 100.0 62 44.0 64 51.0 nan 119.0 nan nan 93
  nan nan nan 123 92 '2008-11-14']
 [53 72 152 31.2 0.7 0.87 61.0 92.0 81 57.0 75 63.0 0.0 127.0 nan nan 86
  nan 3.08 nan 105 75 '2009-07-31']
 [54 85 152 36.8 0.7 0.85 59.0 107.0 66 90.0 120 85.0 0.0 128.0 5.26 21.5
  99 nan 3.22 nan 138 93 '2010-01-20']
 [55 85 152 36.8 0

[[54 83.0 164 30.9 1.3 1.03 51.0 100.0 121 17.0 13 24 42.19 74.0 nan nan
  117 5.4 3.1 nan 147 100 '2006-03-14']
 [55 85.0 165 31.2 1.1 nan 65.0 92.0 85 17.0 23 39 95.6 59.0 nan nan 116
  nan 3.1 nan 176 105 '2007-01-25']
 [55 82.0 165 30.1 1.0 nan 46.0 92.0 122 13.0 17 25 nan nan nan nan 108
  nan nan nan 153 100 '2007-03-14']
 [55 82.0 165 30.1 1.2 nan 48.0 93.0 114 15.0 16 22 nan nan nan nan 121
  nan nan nan 141 91 '2007-04-17']
 [56 78.0 165 28.7 1.1 nan 54.0 134.0 99 15.0 15 18 nan nan nan nan 109
  nan nan nan 144 89 '2007-05-22']
 [56 78.0 165 28.7 1.1 0.93 49.0 139.0 93 14.0 14 24 19.76 127.0 nan nan
  110 nan 3.16 nan 150 100 '2007-11-28']
 [57 78.0 165 28.7 1.2 0.98 63.0 168.0 135 18.0 12 26 nan 68.0 nan nan
  100 nan -3.36 nan 154 102 '2008-05-27']
 [57 78.0 165 28.7 1.1 0.95 58.0 150.0 124 15.0 17 31 28.0 65.0 nan nan
  106 nan 3.19 nan 155 95 '2008-11-20']
 [57 81.0 165 29.8 1.2 0.82 47.0 98.0 143 14.0 17 25 39.0 63.0 nan nan
  103 nan 3.19 nan 142 92 '2009-04-15']
 [58 8

[[59 105.0 152 45.4 0.7 0.8 64.0 90.0 82 12.0 15 25 1.73 68 nan nan 101
  5.6 6.41 nan 124 88 '2006-04-27']
 [59 107.0 152 46.3 0.8 0.81 68.0 87.0 95 14.0 16 28 6.0 69 6.0 20.1 121
  nan 7.46 nan 141 85 '2006-07-18']
 [60 106.0 152 45.9 0.7 0.94 71.0 92.0 122 13.0 19 22 0.0 56 4.1 14.5 114
  5.7 8.04 nan 138 91 '2007-01-18']
 [60 106.0 152 45.9 0.7 0.82 72.0 87.0 119 18.0 21 25 0.0 65 nan nan 108
  5.9 5.67 nan 130 75 '2007-06-20']
 [61 104.0 152 45.0 0.7 0.78 63.0 67.0 86 17.0 19 23 0.0 74 nan nan 110
  5.8 5.41 nan 126 56 '2007-12-21']
 [61 90.0 152 39.0 0.7 0.74 55.0 75.0 95 12.0 13 19 0.0 53 nan nan 102
  5.9 3.3 nan 130 66 '2008-06-26']
 [62 90.0 152 39.0 0.6 0.67 57.0 66.0 90 11.0 14 19 nan 57 nan nan 90 nan
  nan nan 150 79 '2009-01-12']
 [62 92.0 152 39.8 0.7 0.68 63.0 84.0 111 12.0 12 20 0.0 63 nan nan 105
  5.4 3.08 nan 126 80 '2009-06-24']
 [63 96.0 152 41.6 0.8 0.72 56.0 78.0 96 16.0 13 22 0.0 70 3.09 11.9 105
  5.4 3.02 nan 111 56 '2009-12-23']
 [63 98.0 152 42.4 0.6 0.67 

[[56 76 154 32.0 0.8 nan 52 174 160 nan 26 31.0 3.8 52.0 nan nan 97 5.6
  6.89 23.0 150 83 '2011-11-25']
 [57 80 154 33.7 0.8 0.65 49 90 102 25.0 25 24.0 3.57 56.0 nan nan 98 5.5
  4.83 13.0 138 70 '2012-05-25']
 [57 85 154 35.8 0.7 0.75 48 99 128 26.0 30 40.0 0.0 39.0 8.4 31.5 108
  5.5 3.19 nan 126 63 '2012-11-28']
 [58 76 154 32.0 1.0 1.13 42 92 104 45.0 40 nan 17.92 nan 4.55 17.9 103
  5.8 10.1 22.0 104 66 '2013-05-20']]
[[57 75 160 29.3 0.6 0.62 48 140 90 nan 34 37 14.56 74 nan nan 98 5.4
  3.28 17 154 81 '2015-04-20']
 [58 77 160 30.1 0.6 0.69 45 158 70 23.0 41 51 nan 131 4.16 18.3 92 6.2
  3.27 28 152 79 '2015-11-11']
 [58 76 160 29.7 0.6 nan 50 158 82 nan 27 36 18.07 159 4.6 18.1 103 5.6
  3.3 23 152 82 '2016-05-31']]
[[66 75 155 31.2 0.7 0.92 54 104 150 17.0 16 15 nan 48 2.55 12.0 86 5.7
  -3.45 nan 157 89 '2014-04-28']
 [68 71 155 29.6 0.7 1.01 55 103 158 13.0 12 17 0.0 36 2.04 10.1 82 5.6
  3.27 21.0 158 71 '2015-09-28']
 [68 72 155 30.0 0.7 1.14 48 98 207 nan 18 17 10.57 56

[[62 85.0 178 26.8 0.9 0.96 67 86 111 23.0 29 31 5.0 371 nan nan 86 nan
  4.26 nan 156 91 '2007-04-11']
 [62 86.0 178 27.1 0.8 0.72 57 106 149 22.0 26 23 0.0 453 nan nan 92 nan
  3.17 nan 142 87 '2007-10-30']
 [63 88.0 178 27.8 0.9 0.7 60 106 115 25.0 29 21 0.0 353 nan nan 94 nan
  3.3 nan 150 90 '2008-04-30']
 [63 81.0 178 25.6 0.8 0.63 61 97 106 17.0 18 20 2.34 301 nan nan 89 nan
  3.19 nan 137 79 '2008-10-20']
 [64 86.0 178 27.1 0.8 0.69 54 95 134 20.0 22 21 0.0 234 nan nan 88 5.6
  3.19 nan 148 88 '2009-05-25']
 [64 86.0 178 27.1 0.8 0.77 51 97 93 15.0 18 19 7.56 207 2.53 11.0 93 nan
  3.02 nan 151 87 '2009-11-09']
 [65 83.0 178 26.2 0.8 0.66 62 111 102 21.0 25 17 3.0 195 nan nan 101 nan
  3.22 17.8 130 85 '2010-05-07']
 [66 nan 178 nan 0.8 0.72 57 102 120 nan 21 16 0.0 185 nan nan 84 nan nan
  14.9 142 87 '2011-03-28']
 [67 86.0 178 27.1 0.8 0.69 54 102 130 nan 19 16 0.0 202 nan nan 94 5.8
  2.97 32.0 157 92 '2012-04-10']
 [67 81.0 178 25.6 0.8 0.6 57 92 137 18.0 16 17 6.74 263 4.

...

[[34 95 175 31.0 nan 0.78 nan nan nan nan nan nan nan 15 nan nan nan nan
  nan nan 153 101 '2006-04-07']
 [35 88 175 28.7 0.9 nan 43.0 72.0 95.0 20.0 22.0 11.0 5.5 35 nan nan
  97.0 nan nan nan 158 96 '2006-12-22']
 [36 85 175 27.8 0.7 0.7 51.0 127.0 98.0 16.0 17.0 11.0 0.0 15 2.64 11.5
  93.0 nan 3.0 nan 143 82 '2007-11-22']
 [37 92 175 30.0 0.7 0.67 52.0 104.0 69.0 20.0 19.0 12.0 3.43 48 nan nan
  96.0 nan 6.75 nan 140 86 '2008-06-30']
 [37 92 175 30.0 0.8 0.75 43.0 109.0 155.0 21.0 19.0 12.0 0.0 22 nan nan
  91.0 nan -3.19 nan 159 97 '2009-01-30']
 [38 93 175 30.4 0.8 0.79 48.0 134.0 150.0 22.0 24.0 14.0 0.0 35 nan nan
  98.0 nan 3.59 nan 167 104 '2010-01-29']
 [39 95 175 31.0 0.7 0.66 nan nan 121.0 nan 19.0 11.0 6.12 15 5.51 22.1
  101.0 nan 3.36 nan 165 96 '2011-03-28']
 [40 95 175 31.0 0.7 0.75 nan nan 85.0 29.0 28.0 11.0 0.0 26 nan nan 86.0
  5.6 3.34 42.0 136 83 '2011-12-16']
 [41 97 175 31.7 0.7 0.7 47.0 118.0 185.0 nan 25.0 14.0 nan 18 nan nan
  91.0 5.6 3.19 38.0 135 87 '201

[[41 92.0 185 26.9 0.9 0.63 41 103 164 25.0 35 27 10.53 640 3.97 18.7 86
  nan 3.22 nan 123 76 '2010-04-23']
 [41 90.0 185 26.3 1.0 0.59 40 98 137 nan 28 27 15.44 560 6.31 25.8 99
  5.6 3.16 nan 121 80 '2010-11-03']
 [42 88.5 185 25.9 1.1 0.78 43 104 167 nan 27 27 0.0 435 8.08 35.2 93 5.8
  nan 22.0 120 66 '2011-11-28']
 [43 93.0 185 27.2 1.1 0.7 28 92 597 23.0 30 30 3.0 426 7.4 31.1 97 5.9
  3.19 nan 139 78 '2012-05-09']
 [44 84.0 185 24.5 1.2 0.82 40 111 201 34.0 28 23 5.35 399 nan nan 93 5.5
  3.19 30.0 116 66 '2013-06-07']]
[[62 63 157 25.6 1.4 0.93 47 99 125 nan 16 18.0 0 309 nan nan 92 5.4 3.19
  45 109 61 '2012-10-05']
 [63 63 157 25.6 1.4 0.92 49 101 121 18.0 14 nan 0 260 1.69 7.8 88 5.2
  3.19 20 104 56 '2013-08-29']]
[[64 74 152 32.0 0.8 nan nan nan 105 19 27 162 nan 46.0 nan nan 80 6.1
  15.0 nan 104 62 '2015-08-19']
 [64 74 152 32.0 1.0 nan nan nan 189 26 36 200 0.0 nan nan nan 98 nan nan
  nan 103 68 '2015-09-23']]
[[59 75 170 26.0 0.8 1.22 55 185 232 18 29 78 4.0 225 nan 

[[53 75 159 29.7 0.7 0.88 49 132 95 nan 16 20 nan 10 nan nan 93 nan -3.19
  nan 139 77 '2013-07-15']
 [53 75 159 29.7 0.8 0.68 41 84 83 16.0 14 27 1.77 15 2.82 11.2 102 5.5
  3.45 40.0 133 77 '2014-01-24']
 [54 76 159 30.1 0.9 0.69 47 125 91 20.0 20 25 0.0 9 3.08 14.0 89 5.6
  3.44 40.0 137 74 '2014-08-19']
 [54 74 159 29.3 0.8 0.8 43 113 64 23.0 17 21 1.78 15 2.46 11.3 88 5.7
  3.44 28.0 141 77 '2015-02-18']
 [55 77 159 30.5 0.8 0.81 36 128 55 17.0 14 27 2.0 27 nan 8.2 85 5.8 3.27
  nan 134 83 '2015-09-02']
 [55 76 159 30.1 1.0 0.88 34 150 89 nan 18 31 2.62 28 2.17 10.0 88 5.7
  3.27 27.0 123 70 '2016-03-08']]
[[42 80 169 28.0 0.9 0.7 51 138 174 nan 35 47 0.0 504 2.18 9.2 96 5.5
  3.44 23 111 71 '2014-10-20']
 [43 82 169 28.7 0.8 0.73 43 150 143 27.0 37 62 0.0 572 1.74 8.8 80 5.4
  3.27 28 118 74 '2015-09-28']
 [44 83 169 29.1 1.1 0.77 55 154 159 31.0 33 56 nan 714 1.83 8.4 88 5.7
  3.27 15 128 75 '2016-05-03']]
[[56 70 155 29.1 0.7 0.69 56 141 103 15.0 19 10.0 5.97 55.0 nan nan 99
  

  5.7 8.15 47.0 123 84 '2016-03-28']]
[[43 69.0 164 25.7 0.9 0.73 81 107 58 15.0 12 8.0 2.63 36 nan nan 81 nan
  3.1 nan 133 79 '2006-10-11']
 [43 70.0 164 26.0 1.0 0.8 77 127 63 17.0 10 11.0 3.18 54 nan nan 93 nan
  11.1 nan 121 71 '2007-04-11']
 [44 70.0 164 26.0 0.9 0.66 68 119 76 15.0 10 7.0 4.08 22 nan nan 84 nan
  3.0 nan 131 75 '2007-10-18']
 [44 69.0 164 25.7 0.8 0.58 76 118 46 14.0 11 9.0 0.0 15 nan nan 84 nan
  3.3 nan 139 81 '2008-04-14']
 [45 74.0 164 27.5 0.8 0.64 62 122 54 16.0 13 8.0 nan 20 nan nan 94 nan
  3.19 nan 129 78 '2008-10-24']
 [45 77.0 164 28.6 0.9 0.67 67 117 78 14.0 10 10.0 2.88 15 nan nan 86 nan
  3.19 nan 137 79 '2009-04-28']
 [46 74.0 164 27.5 0.9 0.72 57 104 63 12.0 10 8.0 0.0 24 2.77 11.8 95 nan
  3.02 nan 137 78 '2009-11-09']
 [46 74.0 164 27.5 0.8 0.69 56 119 65 15.0 10 8.0 nan 15 2.34 10.9 87 nan
  3.16 12.9 145 89 '2010-05-05']
 [47 74.0 164 27.5 0.9 0.54 55 136 64 nan 7 8.0 nan 15 nan nan 84 nan
  3.16 20.0 132 76 '2010-12-02']
 [48 72.5 164 27.0 0

[[50 90 169 31.5 1.2 0.96 50.0 113.0 118 23.0 27 32 1.25 251 nan nan 97
  nan 3.1 nan 148 95 '2006-03-06']
 [50 92 169 32.2 1.2 1.17 nan nan 142 27.0 32 29 0.0 247 nan nan 110 nan
  3.1 nan 141 92 '2006-09-08']
 [51 91 169 31.9 1.0 1.13 50.0 114.0 115 19.0 29 36 0.0 336 nan nan 99
  nan 3.13 nan 145 89 '2007-03-14']
 [51 92 169 32.2 1.2 1.01 48.0 123.0 128 21.0 30 36 nan 213 nan nan 102
  nan 3.0 nan 135 84 '2007-09-26']
 [52 96 169 33.6 1.1 0.91 53.0 117.0 186 21.0 30 34 0.0 272 nan nan 101
  nan 3.3 nan 141 88 '2008-04-21']
 [52 95 169 33.3 1.1 0.88 39.0 98.0 141 26.0 31 32 5.03 226 4.55 17.7 104
  nan 3.19 nan 134 71 '2008-11-11']
 [53 95 169 33.3 1.1 0.95 50.0 118.0 123 19.0 21 31 nan 229 4.39 16.3 109
  5.5 3.19 nan 131 72 '2009-05-06']
 [53 95 169 33.3 1.2 1.07 44.0 134.0 129 21.0 34 31 nan 280 5.42 20.5 107
  nan 3.02 nan 116 75 '2009-11-06']
 [54 94 169 32.9 1.0 0.91 42.0 129.0 132 19.0 28 27 0.0 279 4.78 17.6 110
  5.6 3.22 nan 148 85 '2010-05-05']
 [55 90 169 31.5 1.1 0.97 48

In [ ]:
print(X_pac[0])
print(X6[:,0].flatten())

In [ ]:
X_pac = np.zeros((5,24))
X_pac[:,:] = np.nan
X_pac[0,:X5.shape[0]] = X5[:,4].flatten() #En la fila 0 y las columnas de X5.shape meteme la variable 4
X_pac[1,:X6.shape[0]] = X6[:,4].flatten()
X_pac[2,:X7.shape[0]] = X7[:,4].flatten()
X_pac[3,:X8.shape[0]] = X8[:,4].flatten()
X_pac[4,:X9.shape[0]] = X9[:,4].flatten()
X_pac

In [ ]:
plt.plot(np.nanmean(X_pac,axis = 0), label = 'no_progresores')
plt.plot(np.nanmean(X_pac,axis = 0) + np.nanstd(X_pac,axis = 0), linestyle = ':', linewidth = 0.5, color = 'k')
plt.plot(np.nanmean(X_pac,axis = 0) - np.nanstd(X_pac,axis = 0), linestyle = ':', linewidth = 0.5, color = 'k')
plt.legend()
#xx = np.arange(X_pac.shape[1])
#plt.plot(xx, X_pac.T, linewidth = 0.3, color = [0.5,0.5,0.5])


In [ ]:
## Analizar las 20.. variables, media y varianza de progresores y no progresores
## funcion que reciba el dataframe y el string de las varibales y que devuelva dos matrices X, una para progresores y no progresores.
## Busco el paciente con mas revisiones y hago una matriz con tantas filas como pacientes y columnas como maximo de revisiones

In [ ]:
df['Historia'][df['Progresion'] == '0']
##DE LA COLUMNA HISTORIA, COJO DE LA COLUMNA DE PROGRESION AQUELLOS QUE SEAN = 0 o 1

In [ ]:
import matplotlib.pyplot as plt

plt.plot(X[:,8],'o-')
plt.plot(X1[:,8]'o-')
plt.plot(X2[:,8],'o-')
plt.plot(X3[:,8],'o-')
plt.plot(X4[:,8],'o-')
plt.plot(X5[:,8],'o-')
plt.plot(X6[:,8],'o-')
plt.plot(X7[:,8],'o-')
plt.plot(X8[:,8],'o-')
plt.plot(X9[:,8],'o-')



In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X[:,8])
X[0]

In [ ]:
X_poly[0]
